<a href="https://colab.research.google.com/github/phenix-project/Colabs/blob/main/alphafold2/Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <center> <b> <font color='black'>  AlphaFold with choice of templates (Phenix version) </font></b> </center>

<font color='green'>AlphaFold can use templates as hypotheses about the structure. This version allows you run AlphaFold with models from the PDB or models you upload. You can run multiple sequences as well.
This notebook is derived from [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) and the DeepMind [AlphaFold2 Colab](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb).
</font>

-----------------
<b> <font color='black'> <center>Instructions:</center>
</font></b> 

1.  Type in a sequence and jobname in the form below and hit the <b><i>Run</i></b> button next to it (little white triangle in black circle) to load this info. You can choose whether and how you want to supply templates

2. Load AlphaFold with the <b><i>Run</i></b> button in the next cell down...takes a couple minutes

3. Run AlphaFold with the <b><i>Run</i></b> button in the last cell...takes 10-20 minutes
</font>


Your model information should download automatically as a zip file at the very end of the third step (when all jobs are done).
</font>
-----------------
<b> <font color='black'> <center>Please cite the ColabFold and AlphaFold2 papers if you use this notebook:</center>
</font></b> 

- <font color='green'>[Mirdita, M.,  Ovchinnikov, S., Steinegger, M.(2021). ColabFold - Making protein folding accessible to all *bioRxiv*, 2021.08.15.456425](https://www.biorxiv.org/content/10.1101/2021.08.15.456425v1)</font> 

- <font color='green'> [Jumper, J., Evans, R., Pritzel, A. et al. Highly accurate protein structure prediction with AlphaFold. Nature 596, 583–589 (2021)](https://www.nature.com/articles/s41586-021-03819-2)
</font>
-----------------


In [ ]:
# USER INPUT SECTION

# IMPORTS, STANDARD PARAMETERS AND METHODS

import os, sys
import os.path
import re
import hashlib
from pathlib import Path
from contextlib import redirect_stderr, redirect_stdout
from io import StringIO
from google.colab import files
import shutil
from string import ascii_uppercase

# Local methods

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

def clear_directories(all_dirs):

  for d in all_dirs:
    if d.exists():
      shutil.rmtree(d)
    d.mkdir(parents=True)


def clean_query(query_sequence):
  query_sequence = "".join(query_sequence.split())
  query_sequence = re.sub(r'[^a-zA-Z]','', query_sequence).upper()
  return query_sequence

def clean_jobname(jobname):
  jobname = "".join(jobname.split())
  jobname = re.sub(r'\W+', '', jobname)
  if len(jobname.split("_")) == 1:
    jobname = add_hash(jobname, query_sequence)
  return jobname

def save_sequence(jobname, query_sequence):
  # save sequence as text file
  filename = f"{jobname}.fasta"
  with open(filename, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)
  print("Saved sequence in %s: %s" %(filename, query_sequence))

def upload_templates(cif_dir):
  manual_templates_uploaded = []
  with redirect_stdout(StringIO()) as out:
    uploaded = files.upload()
    for filename,contents in uploaded.items():
      filepath = Path(cif_dir,filename)
      with filepath.open("w") as fh:
        fh.write(contents.decode("UTF-8"))
        manual_templates_uploaded.append(filepath)
  print("Templates uploaded: %s" %(manual_templates_uploaded))   
  return manual_templates_uploaded

def get_jobnames_sequences_from_file(
    upload_manual_templates = None, cif_dir = None):
  from io import StringIO
  from google.colab import files
  print("Upload file with one jobname, a space and one sequence on each line")
  #with redirect_stdout(StringIO()) as out:
  uploaded = files.upload()
  s = StringIO()
  query_sequences = []
  jobnames = []
  cif_filename_dict = {}
  for filename,contents in uploaded.items():
    print(contents.decode("UTF-8"), file = s)
    text = s.getvalue()
    for line in text.splitlines():
      spl = line.split()
      if len(spl) < 2:
        pass # empty line
      else: # usual
        jobname = spl[0]
        query_sequence = "".join(spl[1:])
        if jobname in jobnames:
          pass # already there
        else:
          query_sequences.append(query_sequence)
          jobnames.append(jobname)
          if upload_manual_templates:
            print("\nPlease upload CIF template for %s" %(jobname))
            sys.stdout.flush()
            cif_filename_dict[jobname] = upload_templates(cif_dir)
  return jobnames, query_sequences, cif_filename_dict

# Set working directory
os.chdir("/content/")

# Clear out directories
parent_dir = Path("/content/manual_templates")
cif_dir = Path(parent_dir,"mmcif")

# GET INPUTS

#@title 1. Enter sequence and jobname. Load with <i><b>Run</b></i> button to left. Repeat to enter any additional sequences.

#@markdown <b><i><font color=green>Protein sequence and job name</font></i></b>

query_sequence = '' #@param {type:"string"}
jobname = '' #@param {type:"string"}

#@markdown <b><i><font color=green>Options (Use PDB templates/upload templates, upload button will show up below if uploading)</font></i></b>
templates_to_use = 'From_PDB' #@param ["From_PDB", "Upload_files", "From_PDB_and_upload_files", "None"]
if templates_to_use == "From_PDB":
  upload_manual_templates = False 
  include_templates_from_pdb = True
elif templates_to_use == "Upload_files":
  include_templates_from_pdb = False
  upload_manual_templates = True
elif templates_to_use == "From_PDB_and_Upload_files":
  include_templates_from_pdb = True
  upload_manual_templates = True
else:
  upload_manual_templates = False 
  include_templates_from_pdb = False

if upload_manual_templates:
  print("Templates will be uploaded")
if include_templates_from_pdb:
  print("Templates from the PDB will be included")

upload_file_with_jobname_space_sequence_lines = False #@param {type:"boolean"}
clear_saved_sequences_and_jobnames = False #@param {type:"boolean"}
maximum_templates_from_pdb = 20 #@param {type:"integer"}


# Initialize query_sequences so we can loop through input
if clear_saved_sequences_and_jobnames or (
     not locals().get('query_sequences', None)):
  query_sequences = []
  jobnames = []
  cif_filename_dict = {}
  clear_directories([parent_dir,cif_dir])

del locals()['clear_saved_sequences_and_jobnames'] # so it updates

if upload_file_with_jobname_space_sequence_lines:
  del locals()['upload_file_with_jobname_space_sequence_lines'] # so it updates
  jobnames, query_sequences, cif_filename_dict = get_jobnames_sequences_from_file()

else: # usual
  if query_sequence and not jobname:
    print("Please enter a job name and rerun")
    raise AssertionError("Please enter a job name and rerun")

  if jobname and not query_sequence:
    print("Please enter a query_sequence and rerun")
    raise AssertionError("Please enter a query_sequence rerun")

  # Add sequence and jobname if new
  if (jobname and query_sequence) and (
       not query_sequence in query_sequences) and (
       not jobname in jobnames):
      query_sequences.append(query_sequence)
      jobnames.append(jobname)
      if upload_manual_templates:
        print("\nPlease upload template for %s" %(jobname))
        sys.stdout.flush()
        cif_filename_dict[jobname] = upload_templates(cif_dir)


# remove whitespaces
for i in range(len(query_sequences)):
  query_sequences[i] = clean_query(query_sequences[i])
  jobnames[i] = clean_jobname(jobnames[i])

  # save the sequence as a file with name jobname.fasta
  save_sequence(jobnames[i], query_sequences[i])
  
print("\nCurrent jobs, sequences, and templates:")

for qs,jn in zip(query_sequences, jobnames):
  template_list = []
  for t in cif_filename_dict.get(jobname,[]):
    template_list.append(os.path.split(str(t))[-1])
  print(jn, qs, template_list)
import sys
sys.stdout.flush()  # seems to overwrite otherwise


if not query_sequences:
  print("Please supply a query sequence and run again")
  raise AssertionError("Need a query sequence")

# STANDARD PARAMETERS AND METHODS

#standard values of parameters
msa_mode = "MMseqs2 (UniRef+Environmental)" 
num_models = 1 
homooligomer = 1
use_msa = True
use_env = True
use_custom_msa = False
use_amber = False 
use_templates = True





In [ ]:
#@title 2. Load AlphaFold2 with the <b><i>Run</i></b> button to the left
%%bash -s $use_amber $use_msa $use_templates
echo "Installing dependencies..."
USE_AMBER=$1
USE_MSA=$2
USE_TEMPLATES=$3

if [ ! -f AF2_READY ]; then
  # install dependencies
  pip -q install biopython dm-haiku ml-collections py3Dmol
  wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py

  # download model
  if [ ! -d "alphafold/" ]; then
    git clone https://github.com/deepmind/alphafold.git --quiet
    (cd alphafold; git checkout 0bab1bf84d9d887aba5cfb6d09af1e8c3ecbc408 --quiet)
    mv alphafold alphafold_
    mv alphafold_/alphafold .
    # remove "END" from PDBs, otherwise biopython complains
    sed -i "s/pdb_lines.append('END')//" /content/alphafold/common/protein.py
    sed -i "s/pdb_lines.append('ENDMDL')//" /content/alphafold/common/protein.py
  fi

  # download model params (~1 min)
  if [ ! -d "params/" ]; then
    mkdir params
    curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar \
    | tar x -C params
  fi
  touch AF2_READY
fi
# download libraries for interfacing with MMseqs2 API
if [ ${USE_MSA} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f MMSEQ2_READY ]; then
    apt-get -qq -y update 2>&1 1>/dev/null
    apt-get -qq -y install jq curl zlib1g gawk 2>&1 1>/dev/null
    touch MMSEQ2_READY
  fi
fi
# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign3=3.2.2 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  (cd /usr/local/lib/python3.7/site-packages; patch -s -p0 < /content/alphafold_/docker/openmm.patch)
  wget -qnc https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
  mv stereo_chemical_props.txt alphafold/common/
  touch AMBER_READY
fi
echo "Done with dependencies"

In [ ]:
#@title 3. Create AlphaFold models with the <b><i>Run</i></b> button to the left

from contextlib import redirect_stderr, redirect_stdout
from dataclasses import dataclass, replace
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO


print("Setting up methods...", end = "")
# setup the model
if "model" not in dir():

  # hiding warning messages
  import warnings
  from absl import logging
  import os
  import tensorflow as tf
  warnings.filterwarnings('ignore')
  logging.set_verbosity("error")
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  tf.get_logger().setLevel('ERROR')

  import sys
  import numpy as np
  import pickle
  from alphafold.common import protein
  from alphafold.data import pipeline
  from alphafold.data import templates
  from alphafold.model import data
  from alphafold.model import config
  from alphafold.model import model
  from alphafold.data.tools import hhsearch
  import colabfold as cf

  # plotting libraries
  import py3Dmol
  import matplotlib.pyplot as plt
  import ipywidgets
  from ipywidgets import interact, fixed, GridspecLayout, Output


if use_amber and "relax" not in dir():
  sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')
  from alphafold.relax import relax

from alphafold.data import mmcif_parsing
from alphafold.data.templates import (_get_pdb_id_and_chain,
                                      _process_single_hit,
                                      _assess_hhsearch_hit,
                                      _build_query_to_hit_index_mapping,
                                      _extract_template_features,
                                      SingleHitResult,
                                      TEMPLATE_FEATURES)

def mk_mock_template(query_sequence):
  # since alphafold's model requires a template input
  # we create a blank example w/ zero input, confidence -1
  ln = len(query_sequence)
  output_templates_sequence = "-"*ln
  output_confidence_scores = np.full(ln,-1)
  templates_all_atom_positions = np.zeros((ln, templates.residue_constants.atom_type_num, 3))
  templates_all_atom_masks = np.zeros((ln, templates.residue_constants.atom_type_num))
  templates_aatype = templates.residue_constants.sequence_to_onehot(output_templates_sequence,
                                                                    templates.residue_constants.HHBLITS_AA_TO_ID)
  template_features = {'template_all_atom_positions': templates_all_atom_positions[None],
                       'template_all_atom_masks': templates_all_atom_masks[None],
                       'template_sequence': [f'none'.encode()],
                       'template_aatype': np.array(templates_aatype)[None],
                       'template_confidence_scores': output_confidence_scores[None],
                       'template_domain_names': [f'none'.encode()],
                       'template_release_date': [f'none'.encode()]}
  return template_features

def mk_template(a3m_lines, template_paths):
  template_featurizer = templates.TemplateHitFeaturizer(
      mmcif_dir=template_paths,
      max_template_date="2100-01-01",
      max_hits=20,
      kalign_binary_path="kalign",
      release_dates_path=None,
      obsolete_pdbs_path=None)

  hhsearch_pdb70_runner = hhsearch.HHSearch(binary_path="hhsearch", databases=[f"{template_paths}/pdb70"])

  hhsearch_result = hhsearch_pdb70_runner.query(a3m_lines)
  hhsearch_hits = pipeline.parsers.parse_hhr(hhsearch_result)
  templates_result = template_featurizer.get_templates(query_sequence=query_sequence,
                                                       query_pdb_code=None,
                                                       query_release_date=None,
                                                       hits=hhsearch_hits)
  return templates_result.features

def set_bfactor(pdb_filename, bfac, idx_res, chains):
  I = open(pdb_filename,"r").readlines()
  O = open(pdb_filename,"w")
  for line in I:
    if line[0:6] == "ATOM  ":
      seq_id = int(line[22:26].strip()) - 1
      seq_id = np.where(idx_res == seq_id)[0][0]
      O.write(f"{line[:21]}{chains[seq_id]}{line[22:60]}{bfac[seq_id]:6.2f}{line[66:]}")
  O.close()

def predict_structure(prefix, feature_dict, Ls, model_params, 
  use_model,
  model_runner_1,
  model_runner_3,
  do_relax=False, random_seed=0):  
  """Predicts structure using AlphaFold for the given sequence."""

  # Minkyung's code
  # add big enough number to residue index to indicate chain breaks
  idx_res = feature_dict['residue_index']
  L_prev = 0
  # Ls: number of residues in each chain
  for L_i in Ls[:-1]:
      idx_res[L_prev+L_i:] += 200
      L_prev += L_i  
  chains = list("".join([ascii_uppercase[n]*L for n,L in enumerate(Ls)]))
  feature_dict['residue_index'] = idx_res

  # Run the models.
  plddts,paes = [],[]
  unrelaxed_pdb_lines = []
  relaxed_pdb_lines = []

  for model_name, params in model_params.items():
    if model_name in use_model:
      print(f"running {model_name}")
      # swap params to avoid recompiling
      # note: models 1,2 have diff number of params compared to models 3,4,5
      if any(str(m) in model_name for m in [1,2]): model_runner = model_runner_1
      if any(str(m) in model_name for m in [3,4,5]): model_runner = model_runner_3
      model_runner.params = params
      
      processed_feature_dict = model_runner.process_features(feature_dict, random_seed=random_seed)
      prediction_result = model_runner.predict(processed_feature_dict)
      unrelaxed_protein = protein.from_prediction(processed_feature_dict,prediction_result)
      unrelaxed_pdb_lines.append(protein.to_pdb(unrelaxed_protein))
      plddts.append(prediction_result['plddt'])
      paes.append(prediction_result['predicted_aligned_error'])

      if do_relax:
        # Relax the prediction.
        amber_relaxer = relax.AmberRelaxation(max_iterations=0,tolerance=2.39,
                                              stiffness=10.0,exclude_residues=[],
                                              max_outer_iterations=20)      
        relaxed_pdb_str, _, _ = amber_relaxer.process(prot=unrelaxed_protein)
        relaxed_pdb_lines.append(relaxed_pdb_str)

  # rerank models based on predicted lddt
  lddt_rank = np.mean(plddts,-1).argsort()[::-1]
  out = {}
  print("reranking models based on avg. predicted lDDT")
  for n,r in enumerate(lddt_rank):
    print(f"model_{n+1} {np.mean(plddts[r])}")

    unrelaxed_pdb_path = f'{prefix}_unrelaxed_model_{n+1}.pdb'    
    with open(unrelaxed_pdb_path, 'w') as f: f.write(unrelaxed_pdb_lines[r])
    set_bfactor(unrelaxed_pdb_path, plddts[r], idx_res, chains)

    if do_relax:
      relaxed_pdb_path = f'{prefix}_relaxed_model_{n+1}.pdb'
      with open(relaxed_pdb_path, 'w') as f: f.write(relaxed_pdb_lines[r])
      set_bfactor(relaxed_pdb_path, plddts[r], idx_res, chains)

    out[f"model_{n+1}"] = {"plddt":plddts[r], "pae":paes[r]}
  return out



def hh_process_seq(query_seq,template_seq,hhDB_dir,db_prefix="DB"):
  """
  This is a hack to get hhsuite output strings to pass on
  to the AlphaFold template featurizer. 
  
  Note: that in the case of multiple templates, this would be faster to build one database for
  all the templates. Currently it builds a database with only one template at a time. Even 
  better would be to get an hhsuite alignment without using a database at all, just between
  pairs of sequence files. However, I have not figured out how to do this.

  Update: I think the hhsearch can be replaced completely, and we can just do a pairwise 
  alignment with biopython, or skip alignment if the seqs match. TODO
  """
  # set up directory for hhsuite DB. Place one template fasta file to be the DB contents
  if hhDB_dir.exists():
    shutil.rmtree(hhDB_dir)
  
  msa_dir = Path(hhDB_dir,"msa")
  msa_dir.mkdir(parents=True)
  template_seq_path = Path(msa_dir,"template.fasta")
  with template_seq_path.open("w") as fh:
    SeqIO.write([template_seq], fh, "fasta")

  # make hhsuite DB
  with redirect_stdout(StringIO()) as out:
    os.chdir(msa_dir)
    %shell ffindex_build -s ../DB_msa.ff{data,index} .
    os.chdir(hhDB_dir)
    %shell ffindex_apply DB_msa.ff{data,index}  -i DB_a3m.ffindex -d DB_a3m.ffdata  -- hhconsensus -M 50 -maxres 65535 -i stdin -oa3m stdout -v 0
    %shell rm DB_msa.ff{data,index}
    %shell ffindex_apply DB_a3m.ff{data,index} -i DB_hhm.ffindex -d DB_hhm.ffdata -- hhmake -i stdin -o stdout -v 0
    %shell cstranslate -f -x 0.3 -c 4 -I a3m -i DB_a3m -o DB_cs219 
    %shell sort -k3 -n -r DB_cs219.ffindex | cut -f1 > sorting.dat

    %shell ffindex_order sorting.dat DB_hhm.ff{data,index} DB_hhm_ordered.ff{data,index}
    %shell mv DB_hhm_ordered.ffindex DB_hhm.ffindex
    %shell mv DB_hhm_ordered.ffdata DB_hhm.ffdata

    %shell ffindex_order sorting.dat DB_a3m.ff{data,index} DB_a3m_ordered.ff{data,index}
    %shell mv DB_a3m_ordered.ffindex DB_a3m.ffindex
    %shell mv DB_a3m_ordered.ffdata DB_a3m.ffdata

  # run hhsearch
  hhsearch_runner = hhsearch.HHSearch(binary_path="hhsearch", databases=[hhDB_dir.as_posix()+"/"+db_prefix])
  with StringIO() as fh:
    SeqIO.write([query_seq], fh, "fasta")
    seq_fasta = fh.getvalue()
  hhsearch_result = hhsearch_runner.query(seq_fasta)

  # process hits
  hhsearch_hits = pipeline.parsers.parse_hhr(hhsearch_result)
  if len(hhsearch_hits) >0:
    hit = hhsearch_hits[0]
    hit = replace(hit,**{"name":template_seq.id})
  else:
    hit = None
  return hit

def plot_plddt_legend():
  thresh = ['plDDT:','Very low (<50)','Low (60)','OK (70)','Confident (80)','Very high (>90)']
  plt.figure(figsize=(1,0.1),dpi=100)
  ########################################
  for c in ["#FFFFFF","#FF0000","#FFFF00","#00FF00","#00FFFF","#0000FF"]:
    plt.bar(0, 0, color=c)
  plt.legend(thresh, frameon=False,
             loc='center', ncol=6,
             handletextpad=1,
             columnspacing=1,
             markerscale=0.5,)
  plt.axis(False)
  return plt

def plot_confidence(outs, model_num=1):
  model_name = f"model_{model_num}"
  plt.figure(figsize=(10,3),dpi=100)
  """Plots the legend for plDDT."""
  #########################################
  plt.subplot(1,2,1); plt.title('Predicted lDDT')
  plt.plot(outs[model_name]["plddt"])
  for n in range(homooligomer+1):
    x = n*(len(query_sequence))
    plt.plot([x,x],[0,100],color="black")
  plt.ylabel('plDDT')
  plt.xlabel('position')
  #########################################
  plt.subplot(1,2,2);plt.title('Predicted Aligned Error')
  plt.imshow(outs[model_name]["pae"], cmap="bwr",vmin=0,vmax=30)
  plt.colorbar()
  plt.xlabel('Scored residue')
  plt.ylabel('Aligned residue')
  #########################################
  return plt

def show_pdb(model_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"model_{model_num}"
  if use_amber:
    pdb_filename = f"{jobname}_relaxed_{model_name}.pdb"
  else:
    pdb_filename = f"{jobname}_unrelaxed_{model_name}.pdb"

  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_filename,'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(homooligomer),list("ABCDEFGH"),
                     ["lime","cyan","magenta","yellow","salmon","white","blue","orange"]):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

def run_job(query_sequence,
        jobname,
        upload_manual_templates,
        manual_templates_uploaded,
        maximum_templates_from_pdb,
        num_models,
        homooligomer,
        use_msa,
        use_env,
        use_custom_msa,
        use_amber,
        use_templates,
        include_templates_from_pdb):

  #@title Get MSA and templates
  print("Getting MSA and templates...")
  if (not include_templates_from_pdb):
    template_paths = None # toss these ... get template_paths later
  if use_templates:
    a3m_lines, template_paths = cf.run_mmseqs2(query_sequence, jobname, use_env, use_templates=True)
    if template_paths is None:
      template_features = mk_mock_template(query_sequence * homooligomer)
    else:
      template_features = mk_template(a3m_lines, template_paths)
  elif use_msa:
    a3m_lines = cf.run_mmseqs2(query_sequence, jobname, use_env)
    template_features = mk_mock_template(query_sequence * homooligomer)
  else:
    template_features = mk_mock_template(query_sequence * homooligomer)
  
  # File for a3m
  a3m_file = f"{jobname}.a3m"

  if use_msa:
    with open(a3m_file, "w") as text_file:
      text_file.write(a3m_lines)
  else:
    a3m_lines = "".join(open(a3m_file,"r").read())
  
  # parse MSA
  msa, deletion_matrix = pipeline.parsers.parse_a3m(a3m_lines)
  
  print("Done with MSA and templates")
  
  #Process templates
  print("PROCESSING TEMPLATES")
  
  os.chdir("/content/")
  
  other_cif_dir = Path("/content/%s" %(template_paths))
  parent_dir = Path("/content/manual_templates")
  cif_dir = Path(parent_dir,"mmcif")
  fasta_dir = Path(parent_dir,"fasta")
  hhDB_dir = Path(parent_dir,"hhDB")
  msa_dir = Path(hhDB_dir,"msa")
  clear_directories([fasta_dir,hhDB_dir,msa_dir])
  
  cif_files = list(cif_dir.glob("*"))
  number_of_supplied_templates = len(cif_files)
  # Only include the cif_files in manual_templates_uploaded
  manual_files_as_text = []
  for f in manual_templates_uploaded:
    manual_files_as_text.append(
        os.path.split(str(f))[-1])
  cif_files_to_include = []
  for cif_file in cif_files:
    text = os.path.split(str(cif_file))[-1]
    if text in manual_files_as_text:
      cif_files_to_include.append(cif_file)
  cif_files = cif_files_to_include
      
  if include_templates_from_pdb:
    other_cif_files = list(other_cif_dir.glob("*"))
    cif_files += other_cif_files
  print("CIF files to include:",cif_files)
  query_seq = SeqRecord(Seq(query_sequence),id="query",name="",description="")
  query_seq_path = Path(fasta_dir,"query.fasta")
  with query_seq_path.open("w") as fh:
      SeqIO.write([query_seq], fh, "fasta")
  
  shutil.copyfile(query_seq_path,Path(msa_dir,"query.fasta"))
  seqs = []
  template_hit_list = []
  
  n_used = 0
  for i,filepath in enumerate(cif_files):
    if not str(filepath).endswith(".cif"): continue
    if n_used >= maximum_templates_from_pdb + number_of_supplied_templates:
      continue
    n_used += 1
    print("CIF file included:",i+1,str(filepath))
    with filepath.open("r") as fh:
      filestr = fh.read()
      mmcif_obj = mmcif_parsing.parse(file_id=filepath.stem,mmcif_string=filestr)
      mmcif = mmcif_obj.mmcif_object
      if not mmcif: continue
  
      for chain_id,template_sequence in mmcif.chain_to_seqres.items():
        template_sequence = mmcif.chain_to_seqres[chain_id]
        seq_name = filepath.stem.upper()+"_"+chain_id
        seq = SeqRecord(Seq(template_sequence),id=seq_name,name="",description="")
        seqs.append(seq)
  
        with  Path(fasta_dir,seq.id+".fasta").open("w") as fh:
          SeqIO.write([seq], fh, "fasta")
  
        """
        At this stage, we have a template sequence.
        and a query sequence. 
        There are two options to generate template features:
          1. Write new code to manually generate template features
          2. Get an hhr alignment string, and pass that
            to the existing template featurizer. 
            
        I chose the second, implemented in hh_process_seq()
        """
        SeqIO.write([seq], sys.stdout, "fasta")
        SeqIO.write([query_seq], sys.stdout, "fasta")
        try:
          hit = hh_process_seq(query_seq,seq,hhDB_dir)
        except Exception as e:
          hit = None
        if hit is not None:
          template_hit_list.append(hit)
  
  if template_hit_list:
    #process hits into template features
    template_hit_list = [replace(hit,**{"index":i+1}) for i,hit in enumerate(template_hit_list)]
  
  if (len(manual_templates_uploaded) > 0) and upload_manual_templates and (not template_hit_list):
    # check to make sure we got something
    # need template and did not get any
      print("\n",80*"-")
      print("\nNo templates obtained...please be sure to use a .cif file")
      print("Use this converter: https://mmcif.pdbj.org/converter/")
      print("\nYou can hit the red run button and load a new file")
      print("\nYou can then hit all the remaining red run buttons one by one")
      print("\n ... or you can go up to Runtime and hit 'Run all' again to start over")
      print("\n",80*"-")
      raise AssertionError("Failed to read template file")
  elif use_templates and template_hit_list:
    # have new templates to work with
  
    template_features = {}
    for template_feature_name in TEMPLATE_FEATURES:
      template_features[template_feature_name] = []
  
    for i,hit in enumerate(sorted(template_hit_list, key=lambda x: x.sum_probs, reverse=True)):
      # modifications to alphafold/data/templates.py _process_single_hit
      hit_pdb_code, hit_chain_id = _get_pdb_id_and_chain(hit)
      mapping = _build_query_to_hit_index_mapping(
      hit.query, hit.hit_sequence, hit.indices_hit, hit.indices_query,
      query_sequence)
      template_sequence = hit.hit_sequence.replace('-', '')
  
      try:
        features, realign_warning = _extract_template_features(
          mmcif_object=mmcif,
          pdb_id=hit_pdb_code,
          mapping=mapping,
          template_sequence=template_sequence,
          query_sequence=query_sequence,
          template_chain_id=hit_chain_id,
          kalign_binary_path="kalign")
      except Exception as e:
        continue
      features['template_sum_probs'] = [hit.sum_probs]
  
      single_hit_result = SingleHitResult(features=features, error=None, warning=None)
      for k in template_features:
        template_features[k].append(features[k])
  
    for name in template_features:
      template_features[name] = np.stack(
          template_features[name], axis=0).astype(TEMPLATE_FEATURES[name])
      
    #overwrite template data
    template_paths = cif_dir.as_posix()


    # Select only one chain from any cif file
    unique_template_hits = []
    pdb_text_list = []
    for hit in template_hit_list:
      pdb_text = hit.name.split()[0].split("_")[0]
      if not pdb_text in pdb_text_list:
        pdb_text_list.append(pdb_text)
        unique_template_hits.append(hit)
    template_hit_list = unique_template_hits
    template_hits = template_hit_list

    print("\nIncluding templates:")
    for hit in template_hit_list:
      print("\t",hit.name.split()[0])
    if len(template_hit_list) == 0:
      print("No templates found...quitting")
      raise AssertionError("No templates found...quitting")
    os.chdir("/content/")
  
    for key,value in template_features.items():
      if np.all(value==0):
        print("ERROR: Some template features are empty")
  else:  # no templates
    print("Not using any templates")
  
  print("\nPREDICTING STRUCTURE")

  
  # collect model weights
  use_model = {}
  model_params = {}
  model_runner_1 = None
  model_runner_3 = None
  for model_name in ["model_1","model_2","model_3","model_4","model_5"][:num_models]:
    use_model[model_name] = True
    if model_name not in list(model_params.keys()):
      model_params[model_name] = data.get_model_haiku_params(model_name=model_name+"_ptm", data_dir=".")
      if model_name == "model_1":
        model_config = config.model_config(model_name+"_ptm")
        model_config.data.eval.num_ensemble = 1
        model_runner_1 = model.RunModel(model_config, model_params[model_name])
      if model_name == "model_3":
        model_config = config.model_config(model_name+"_ptm")
        model_config.data.eval.num_ensemble = 1
        model_runner_3 = model.RunModel(model_config, model_params[model_name])
  if homooligomer == 1:
    msas = [msa]
    deletion_matrices = [deletion_matrix]
  else:
    # make multiple copies of msa for each copy
    # AAA------
    # ---AAA---
    # ------AAA
    #
    # note: if you concat the sequences (as below), it does NOT work
    # AAAAAAAAA
    msas = []
    deletion_matrices = []
    Ln = len(query_sequence)
    for o in range(homooligomer):
      L = Ln * o
      R = Ln * (homooligomer-(o+1))
      msas.append(["-"*L+seq+"-"*R for seq in msa])
      deletion_matrices.append([[0]*L+mtx+[0]*R for mtx in deletion_matrix])
  
  # gather features
  feature_dict = {
      **pipeline.make_sequence_features(sequence=query_sequence*homooligomer,
                                        description="none",
                                        num_res=len(query_sequence)*homooligomer),
      **pipeline.make_msa_features(msas=msas,deletion_matrices=deletion_matrices),
      **template_features
  }
  outs = predict_structure(jobname, feature_dict,
                           Ls=[len(query_sequence)]*homooligomer,
                           model_params=model_params, use_model=use_model,
                           model_runner_1=model_runner_1,
                           model_runner_3=model_runner_3,
                           do_relax=use_amber)
  print("DONE WITH STRUCTURE")
  
  #@title Making plots...
  
  # gather MSA info
  deduped_full_msa = list(dict.fromkeys(msa))
  msa_arr = np.array([list(seq) for seq in deduped_full_msa])
  seqid = (np.array(list(query_sequence)) == msa_arr).mean(-1)
  seqid_sort = seqid.argsort() #[::-1]
  non_gaps = (msa_arr != "-").astype(float)
  non_gaps[non_gaps == 0] = np.nan
  
  ##################################################################
  plt.figure(figsize=(14,4),dpi=100)
  ##################################################################
  plt.subplot(1,2,1); plt.title("Sequence coverage")
  plt.imshow(non_gaps[seqid_sort]*seqid[seqid_sort,None],
             interpolation='nearest', aspect='auto',
             cmap="rainbow_r", vmin=0, vmax=1, origin='lower')
  plt.plot((msa_arr != "-").sum(0), color='black')
  plt.xlim(-0.5,msa_arr.shape[1]-0.5)
  plt.ylim(-0.5,msa_arr.shape[0]-0.5)
  plt.colorbar(label="Sequence identity to query",)
  plt.xlabel("Positions")
  plt.ylabel("Sequences")
  
  ##################################################################
  plt.subplot(1,2,2); plt.title("Predicted lDDT per position")
  for model_name,value in outs.items():
    plt.plot(value["plddt"],label=model_name)
  if homooligomer > 0:
    for n in range(homooligomer+1):
      x = n*(len(query_sequence)-1)
      plt.plot([x,x],[0,100],color="black")
  plt.legend()
  plt.ylim(0,100)
  plt.ylabel("Predicted lDDT")
  plt.xlabel("Positions")
  plt.savefig(jobname+"_coverage_lDDT.png")
  ##################################################################
  plt.show()
  
  print("Predicted Alignment Error")
  ##################################################################
  plt.figure(figsize=(3*num_models,2), dpi=100)
  for n,(model_name,value) in enumerate(outs.items()):
    plt.subplot(1,num_models,n+1)
    plt.title(model_name)
    plt.imshow(value["pae"],label=model_name,cmap="bwr",vmin=0,vmax=30)
    plt.colorbar()
  plt.savefig(jobname+"_PAE.png")
  plt.show()
  ##################################################################
  #@title Displaying 3D structure... {run: "auto"}
  model_num = 1 
  color = "lDDT" 
  show_sidechains = False 
  show_mainchains = False 
  
  
  
  show_pdb(model_num,show_sidechains, show_mainchains, color).show()
  if color == "lDDT": plot_plddt_legend().show()  
  plot_confidence(outs, model_num).show()
  #@title Packaging and downloading results...
  
  #@markdown When modeling is complete .zip files with results will be downloaded automatically.
  
  citations = {
  "Mirdita2021":  """@article{Mirdita2021,
  author = {Mirdita, Milot and Ovchinnikov, Sergey and Steinegger, Martin},
  doi = {10.1101/2021.08.15.456425},
  journal = {bioRxiv},
  title = {{ColabFold - Making Protein folding accessible to all}},
  year = {2021},
  comment = {ColabFold including MMseqs2 MSA server}
  }""",
    "Mitchell2019": """@article{Mitchell2019,
  author = {Mitchell, Alex L and Almeida, Alexandre and Beracochea, Martin and Boland, Miguel and Burgin, Josephine and Cochrane, Guy and Crusoe, Michael R and Kale, Varsha and Potter, Simon C and Richardson, Lorna J and Sakharova, Ekaterina and Scheremetjew, Maxim and Korobeynikov, Anton and Shlemov, Alex and Kunyavskaya, Olga and Lapidus, Alla and Finn, Robert D},
  doi = {10.1093/nar/gkz1035},
  journal = {Nucleic Acids Res.},
  title = {{MGnify: the microbiome analysis resource in 2020}},
  year = {2019},
  comment = {MGnify database}
  }""",
    "Eastman2017": """@article{Eastman2017,
  author = {Eastman, Peter and Swails, Jason and Chodera, John D. and McGibbon, Robert T. and Zhao, Yutong and Beauchamp, Kyle A. and Wang, Lee-Ping and Simmonett, Andrew C. and Harrigan, Matthew P. and Stern, Chaya D. and Wiewiora, Rafal P. and Brooks, Bernard R. and Pande, Vijay S.},
  doi = {10.1371/journal.pcbi.1005659},
  journal = {PLOS Comput. Biol.},
  number = {7},
  title = {{OpenMM 7: Rapid development of high performance algorithms for molecular dynamics}},
  volume = {13},
  year = {2017},
  comment = {Amber relaxation}
  }""",
    "Jumper2021": """@article{Jumper2021,
  author = {Jumper, John and Evans, Richard and Pritzel, Alexander and Green, Tim and Figurnov, Michael and Ronneberger, Olaf and Tunyasuvunakool, Kathryn and Bates, Russ and {\v{Z}}{\'{i}}dek, Augustin and Potapenko, Anna and Bridgland, Alex and Meyer, Clemens and Kohl, Simon A. A. and Ballard, Andrew J. and Cowie, Andrew and Romera-Paredes, Bernardino and Nikolov, Stanislav and Jain, Rishub and Adler, Jonas and Back, Trevor and Petersen, Stig and Reiman, David and Clancy, Ellen and Zielinski, Michal and Steinegger, Martin and Pacholska, Michalina and Berghammer, Tamas and Bodenstein, Sebastian and Silver, David and Vinyals, Oriol and Senior, Andrew W. and Kavukcuoglu, Koray and Kohli, Pushmeet and Hassabis, Demis},
  doi = {10.1038/s41586-021-03819-2},
  journal = {Nature},
  pmid = {34265844},
  title = {{Highly accurate protein structure prediction with AlphaFold.}},
  year = {2021},
  comment = {AlphaFold2 + BFD Database}
  }""",
    "Mirdita2019": """@article{Mirdita2019,
  author = {Mirdita, Milot and Steinegger, Martin and S{\"{o}}ding, Johannes},
  doi = {10.1093/bioinformatics/bty1057},
  journal = {Bioinformatics},
  number = {16},
  pages = {2856--2858},
  pmid = {30615063},
  title = {{MMseqs2 desktop and local web server app for fast, interactive sequence searches}},
  volume = {35},
  year = {2019},
  comment = {MMseqs2 search server}
  }""",
    "Steinegger2019": """@article{Steinegger2019,
  author = {Steinegger, Martin and Meier, Markus and Mirdita, Milot and V{\"{o}}hringer, Harald and Haunsberger, Stephan J. and S{\"{o}}ding, Johannes},
  doi = {10.1186/s12859-019-3019-7},
  journal = {BMC Bioinform.},
  number = {1},
  pages = {473},
  pmid = {31521110},
  title = {{HH-suite3 for fast remote homology detection and deep protein annotation}},
  volume = {20},
  year = {2019},
  comment = {PDB70 database}
  }""",
    "Mirdita2017": """@article{Mirdita2017,
  author = {Mirdita, Milot and von den Driesch, Lars and Galiez, Clovis and Martin, Maria J. and S{\"{o}}ding, Johannes and Steinegger, Martin},
  doi = {10.1093/nar/gkw1081},
  journal = {Nucleic Acids Res.},
  number = {D1},
  pages = {D170--D176},
  pmid = {27899574},
  title = {{Uniclust databases of clustered and deeply annotated protein sequences and alignments}},
  volume = {45},
  year = {2017},
  comment = {Uniclust30/UniRef30 database},
  }""",
    "Berman2003": """@misc{Berman2003,
  author = {Berman, Helen and Henrick, Kim and Nakamura, Haruki},
  booktitle = {Nat. Struct. Biol.},
  doi = {10.1038/nsb1203-980},
  number = {12},
  pages = {980},
  pmid = {14634627},
  title = {{Announcing the worldwide Protein Data Bank}},
  volume = {10},
  year = {2003},
  comment = {templates downloaded from wwPDB server}
  }""",
  }
  
  to_cite = [ "Mirdita2021", "Jumper2021" ]
  if use_msa:       to_cite += ["Mirdita2019"]
  if use_msa:       to_cite += ["Mirdita2017"]
  if use_env:       to_cite += ["Mitchell2019"]
  if use_templates: to_cite += ["Steinegger2019"]
  if use_templates: to_cite += ["Berman2003"]
  if use_amber:     to_cite += ["Eastman2017"]
  
  with open(f"{jobname}.bibtex", 'w') as writer:
    for i in to_cite:
      writer.write(citations[i])
      writer.write("\n")
  
  print(f"Found {len(to_cite)} citation{'s' if len(to_cite) > 1 else ''} for tools or databases.")
  if use_custom_msa:
    print("Don't forget to cite your custom MSA generation method.")
  
  !echo 'FILES TO PACKAGE: $a3m_file $jobname"_"*"relaxed_model_"*".pdb" $jobname"_coverage_lDDT.png" $jobname".bibtex" $jobname"_PAE.png" '
  try:
    print("zipping files...")
    !zip -FSr $jobname".result.zip" $a3m_file $jobname"_"*"relaxed_model_"*".pdb" $jobname"_coverage_lDDT.png" $jobname".bibtex" $jobname"_PAE.png"
  except Exception as e:
    print("unable to zip files")

  filename = f"{jobname}.result.zip"
  if os.path.isfile(filename):
    print("About to download %s" %(filename))
  
    try:
      print("Downloading zip file %s" %(filename))
      files.download(filename)
      print("Start of download successful (NOTE: if the download symbol does not go away it did not work. Download it manually using the folder icon to the left)")
      return filename
    except Exception as e:
      print("Unable to download zip file %s" %(filename))
      return None
  else:
    print("No .zip file %s created" %(filename))
    return None

# RUN THE JOBS HERE

for query_sequence, jobname in zip(query_sequences, jobnames):
  print("\n","****************************************","\n",
         "RUNNING JOB %s with sequence %s\n" %(
    jobname, query_sequence),
    "****************************************","\n")
  # GET TEMPLATES AND SET UP FILES



  # User input of manual templates
  manual_templates_uploaded = cif_filename_dict.get(
      jobname,[])
  if manual_templates_uploaded:
    print("Using uploaded templates %s for this run" %(
        manual_templates_uploaded))

  try:
    filename = run_job(query_sequence,
        jobname,
        upload_manual_templates,
        manual_templates_uploaded,
        maximum_templates_from_pdb,
        num_models,
        homooligomer,
        use_msa,
        use_env,
        use_custom_msa,
        use_amber,
        use_templates,
        include_templates_from_pdb)
    if filename:
      print("FINISHED JOB (%s) %s with sequence %s\n" %(
        filename, jobname, query_sequence),
        "****************************************","\n")
    else:
      print("NO RESULT FOR JOB %s with sequence %s\n" %(
    jobname, query_sequence),
    "****************************************","\n")

  except Exception as e:
    print("FAILED: JOB %s with sequence %s\n\n%s\n" %(
    jobname, query_sequence, str(e)),
    "****************************************","\n")


print("\nDOWNLOADING FILES NOW:\n")
for query_sequence, jobname in zip(query_sequences, jobnames):
  filename = f"{jobname}.result.zip"
  if os.path.isfile(filename):
    print(filename)

print("\nALL DONE\n")
  


**Helpful hints**

* To upload
a file with a jobname and a sequence on each line, 
check ***upload_file_with_jobname_space_sequence_lines*** and hit
the ***Run*** button to the left of the first cell

* To start over, check ***clear_saved_sequences_and_jobnames***.

* You can step through this notebook one part at a time
by hitting the ***Run*** buttons to the left one at a time. 

* The cell that is active is indicated by a ***Run*** button that has turned into a black circle with a moving black arc

* When execution is done, the ***Run*** button will go back 
to its original white triangle inside a black circle

* You can stop execution of the active cell by hitting its ***Run*** button. It will turn red to indicate it has stopped.

* You can rerun any cell any time that nothing is running.  That means you can go all the way through, then go back to the first cell and enter another sequence and redo the procedure.

* If something goes wrong, the Colab Notebook will print out
an error message.  Usually this will be something telling you
how to change your inputs.  You enter your new inputs and
hit the ***Run*** button again to carry on.

* The automatic download may not always work. Normally the
file download starts when the .zip files are created,
but the actual download happens when all the AlphaFold
models are completed.
You can click on the 
folder icon to the left of the window and download your
jobname.zip file manually.  Open and close the file
broswer to show recently-added files.

* Your Colab connection may time out if you go away and
leave it, or if you run for a long time (12 hr).
If your connection times out you lose everything that
is not yet downloaded. So you might want to download as you go
if you are running multiple sequences, or else run in batches
that are not too large (perhaps up to 10 at a time).


* The zip file or files will usually not be automatically downloaded until the very end of the job. If you are running multiple sequences you might want to manually download results as they appear so they are safe.
* Your sequence should contain only the 1-letter code of one protein chain. It can contain spaces if you want.
* If you upload a file with multiple sequences, each line of the file should have exactly one job name, a space, and a sequence, like this:

7n8i_24237 VIWMTQSPSSLSASVGDRVTITCQASQDIRFYLNWYQQKPGKAPKLLISDASNMETGVPSRFSGS

7lvr_23541 MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFSETG

* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can instead manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Result zip file contents**

1. PDB formatted structure
2. Plot of the model quality (IDDT).
3. Plots of the MSA coverage.
4. A3M formatted input MSA.
5. BibTeX file with citations for all used tools and databases.

At the end of the job the `jobname.result.zip` file or files will be downloaded automatically.


**Colab limitations**
* While Colab is free, it is designed for interactive work and not-unlimited memory and GPU usage. It will time-out after a few hours and it may check that you are not a robot at random times.  On a time-out you may lose your work. You can increase your allowed time with Colab+

* AlphaFold can crash if it requires too much memory. On a crash you may lose all your work that is not yet downloaded. You can have more memory accessible if you have Colab+. If you are familiar with Colab scripts you can try this [hack](https://towardsdatascience.com/double-your-google-colab-ram-in-10-seconds-using-these-10-characters-efa636e646ff ) to increase your memory allowance.


**Description of the plots**

*   **Number of sequences per position** - Look for at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.


**Acknowledgments**

- <b> <font color='green'>This notebook is based on the very nice notebook from ColabFold ([Mirdita et al., *bioRxiv*, 2021](https://www.biorxiv.org/content/10.1101/2021.08.15.456425v1), https://github.com/sokrypton/ColabFold)</font></b> 

- <b><font color='green'>ColabFold is based on AlphaFold2 [(Jumper et al. 2021)](https://www.nature.com/articles/s41586-021-03819-2)
</font></b>